<a href="https://colab.research.google.com/github/Mahmuda-104/Thesis/blob/main/CNN_New1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
import os
from scipy.io import loadmat

In [18]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
data_path = '/content/drive/My Drive/Data'

In [21]:
import os
import scipy.io
import numpy as np
from scipy import signal

lowcut = 0.5 
highcut = 100  
fs_original = 500
fs_new = 250

control_data = []
control_path = os.path.join(data_path, 'Control')
for foldername in os.listdir(control_path):
    subfolder_path = os.path.join(control_path, foldername)
    control_data.append(subfolder_path)

Control30 = []
for control_data_path in control_data:
    n_epochs = 55 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    control_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{control_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        
        b, a = signal.butter(4, [lowcut, highcut], fs=fs_original, btype='band')
        filtered_data = signal.filtfilt(b, a, mat_data1, axis=-1)
        #filtered_data=mat_data1
        
        # Downsampling
        num_samples_original = filtered_data.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(filtered_data, num_samples_new, axis=-1)
        
        control_arr.append(np.array(downsampled_data))
        
    Control30.append(np.array(control_arr))

Control30 = np.array(Control30)
print( Control30[0].shape)


(55, 60, 1000)


In [22]:
print("Filtered and downsampled data shape:", Control30.shape)

Filtered and downsampled data shape: (32, 55, 60, 1000)


In [23]:
import os
import scipy.io
import numpy as np
from scipy import signal

lowcut = 0.5 
highcut = 100  
fs_original = 500
fs_new = 250  

concussed_data = []
concussed_path = os.path.join(data_path, 'Concussed')
for foldername in os.listdir(concussed_path):
    subfolder_path = os.path.join(concussed_path, foldername)
    concussed_data.append(subfolder_path)

Concussed49 = []
for concussed_data_path in concussed_data:
    n_epochs = 55 #len([f for f in os.listdir(control_data_path) if f.endswith('.mat')])
    concussed_arr = []
    for i in range(1, n_epochs+1):
        epoch_path = f"{concussed_data_path}/trial{i}.mat"
        mat_data = scipy.io.loadmat(epoch_path)
        mat_data1 = mat_data["trialData_i"]
        
        b, a = signal.butter(4, [lowcut, highcut], fs=fs_original, btype='band')
        filtered_data = signal.filtfilt(b, a, mat_data1, axis=-1)
        # filtered_data=mat_data1
        
        # Downsampling
        num_samples_original = filtered_data.shape[-1]
        num_samples_new = int(num_samples_original * fs_new / fs_original)
        downsampled_data = signal.resample(filtered_data, num_samples_new, axis=-1)
        
        concussed_arr.append(np.array(downsampled_data))
        
    Concussed49.append(np.array(concussed_arr))

Concussed49 = np.array(Concussed49)
print( Concussed49[0].shape)


(55, 60, 1000)


In [24]:
print("Filtered and downsampled data shape:", Concussed49.shape)

Filtered and downsampled data shape: (52, 55, 60, 1000)


In [25]:
import numpy as np
from sklearn.model_selection import train_test_split
dataset = np.concatenate((Control30, Concussed49), axis=0)

#(0 for control, 1 for concussed)
labels = np.concatenate((np.zeros(len(Control30)), np.ones(len(Concussed49))))

In [26]:
train_ratio = 0.8  # 80% for training
val_ratio = 0.1   # 10% for validation
test_ratio = 0.1  # 10% for testing

In [27]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

In [29]:
import tensorflow as tf
from tensorflow.keras import layers

# Normalize the input data (if required)
# X_train = X_train / 255.0
# X_val = X_val / 255.0
# X_test = X_test / 255.0

# Reshape the input data based on the expected input shape of the model
input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
X_train = X_train.reshape((-1,) + input_shape)
X_val = X_val.reshape((-1,) + input_shape)
X_test = X_test.reshape((-1,) + input_shape)

# Build the CNN model
model = tf.keras.Sequential()
model.add(layers.Conv2D(16, kernel_size=(6, 6),strides=2, activation='relu',padding='same', input_shape=input_shape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', strides=1,padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu', strides=1,padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))  
from keras.callbacks import EarlyStopping


callback_list = EarlyStopping(monitor = 'val_loss',
                              patience = 20,
                              restore_best_weights=True)


learning_rate = 0.005  # Specify your desired learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
val_accuracy=model.evaluate(X_val,y_val)

# Print the test accuracy
print("Test Accuracy:", test_accuracy)
print("Validation Accuracy", val_accuracy)

Epoch 1/20
3/3 [==============================] - 14s 4s/step - loss: 145.5286 - accuracy: 0.3939 - val_loss: 159.1699 - val_accuracy: 0.3333
Epoch 2/20
3/3 [==============================] - 12s 4s/step - loss: 927.5649 - accuracy: 0.5303 - val_loss: 30.5675 - val_accuracy: 0.5556
Epoch 3/20
3/3 [==============================] - 11s 3s/step - loss: 1301.8198 - accuracy: 0.6212 - val_loss: 100.6164 - val_accuracy: 0.3333
Epoch 4/20
3/3 [==============================] - 11s 3s/step - loss: 459.6374 - accuracy: 0.5000 - val_loss: 39.8417 - val_accuracy: 0.6667
Epoch 5/20
3/3 [==============================] - 11s 3s/step - loss: 664.8944 - accuracy: 0.5152 - val_loss: 51.7018 - val_accuracy: 0.6667
Epoch 6/20
3/3 [==============================] - 12s 4s/step - loss: 1133.9039 - accuracy: 0.4545 - val_loss: 41.9475 - val_accuracy: 0.6667
Epoch 7/20
3/3 [==============================] - 12s 4s/step - loss: 664.4165 - accuracy: 0.6667 - val_loss: 40.3202 - val_accuracy: 0.6667
Epoch 8/2

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers
# from sklearn.model_selection import train_test_split

# # Split the dataset into training, validation, and test sets
# X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

# # Normalize the input data (if required)
# # X_train = X_train / 255.0
# # X_val = X_val / 255.0
# # X_test = X_test / 255.0

# # Reshape the input data based on the expected input shape of the model
# input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
# X_train = X_train.reshape((-1,) + input_shape)
# X_val = X_val.reshape((-1,) + input_shape)
# X_test = X_test.reshape((-1,) + input_shape)

# # Build the CNN model
# model = tf.keras.Sequential()
# model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))

# # Compile the model with the desired loss function and optimizer
# learning_rate = 0.05  # Specify your desired learning rate
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# # Train the model
# history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

# # Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(X_test, y_test)

# # Print the test accuracy
# print("Test Accuracy:", test_accuracy)


In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers
# from sklearn.model_selection import train_test_split

# # Split the dataset into training, validation, and test sets
# X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=test_ratio, stratify=labels, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio/(train_ratio+val_ratio), stratify=y_train, random_state=42)

# # Normalize the input data (if required)
# # X_train = X_train / 255.0
# # X_val = X_val / 255.0
# # X_test = X_test / 255.0

# # Reshape the input data based on the expected input shape of the model
# input_shape = X_train.shape[1:]  # Assuming the first dimension is the number of samples
# X_train = X_train.reshape((-1,) + input_shape)
# X_val = X_val.reshape((-1,) + input_shape)
# X_test = X_test.reshape((-1,) + input_shape)

# # Build the CNN model
# model = tf.keras.Sequential()
# model.add(layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(layers.MaxPooling2D(pool_size=(2, 2)))
# model.add(layers.Flatten())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dense(1, activation='linear'))

# # Compile the model with the desired loss function and optimizer
# learning_rate = 0.05  # Specify your desired learning rate
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
# model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

# # Train the model
# history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32)

# # Evaluate the model on the test set
# test_loss, test_accuracy = model.evaluate(X_test, y_test)

# # Print the test accuracy
# print("Test Accuracy:", test_accuracy)
